> **In this notebook, I have built a simple multilayer perceptron (MLP) from scratch. I took inspiration from the Neural Probabilistic Language Model (NPLM) paper, which introduced the idea of using neural networks to learn distributed representations of words and predict the next word in a sequence.The notebook covers:**

```python
# Implementing the MLP architecture for character-level language modeling.
# Initializing and training the model on a small dataset.
# Experimenting with hyperparameters such as learning rate and model size.
# Evaluating training and validation loss.
# Observing the effects of overfitting and underfitting.
```
> This hands-on implementation deepens understanding of how neural networks learn and generalize from data.


In [1]:
# imports
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(words[20:25] ,'\n[ Num. of words]:',[len(words)])

['victoria', 'madison', 'luna', 'grace', 'chloe'] 
[ Num. of words]: [32033]


In [4]:
# build vocabulary of characters and map to int.
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [5]:
# build the dataset
block_size = 3 # how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # aaghe badho

X = torch.tensor(X)
Y = torch.tensor(Y)

In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [7]:
C = torch.randn((27, 2)) # token embedding
emb = C[X] ; emb.shape

torch.Size([32, 3, 2])

In [8]:
W1 = torch.randn((6, 100)) # 6 inputs (3 chars -> 2 vals) => 100 neurons
b1 = torch.randn(100)

In [10]:
emb.view(-1,6)

tensor([[ 0.5543,  1.6604,  0.5543,  1.6604,  0.5543,  1.6604],
        [ 0.5543,  1.6604,  0.5543,  1.6604, -0.2901, -1.5833],
        [ 0.5543,  1.6604, -0.2901, -1.5833,  1.3519,  0.3837],
        [-0.2901, -1.5833,  1.3519,  0.3837,  1.3519,  0.3837],
        [ 1.3519,  0.3837,  1.3519,  0.3837, -0.6418,  1.0960],
        [ 0.5543,  1.6604,  0.5543,  1.6604,  0.5543,  1.6604],
        [ 0.5543,  1.6604,  0.5543,  1.6604, -1.0759,  2.3301],
        [ 0.5543,  1.6604, -1.0759,  2.3301,  0.5395, -2.2043],
        [-1.0759,  2.3301,  0.5395, -2.2043, -1.0781, -0.1733],
        [ 0.5395, -2.2043, -1.0781, -0.1733,  0.5595, -0.2312],
        [-1.0781, -0.1733,  0.5595, -0.2312, -1.0781, -0.1733],
        [ 0.5595, -0.2312, -1.0781, -0.1733, -0.6418,  1.0960],
        [ 0.5543,  1.6604,  0.5543,  1.6604,  0.5543,  1.6604],
        [ 0.5543,  1.6604,  0.5543,  1.6604, -0.6418,  1.0960],
        [ 0.5543,  1.6604, -0.6418,  1.0960,  0.5595, -0.2312],
        [-0.6418,  1.0960,  0.5595, -0.2

In [11]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # layer 1 o/p vals

In [12]:
# sigmoid function weights and bias => i/p 100 vals from layer 1 & o/p 27 probs for 27 chars
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [13]:
logits = h @ W2 + b2
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
prob.shape

torch.Size([32, 27])

In [15]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(21.5245)

In [16]:
# funtion for building complete dataset for splitting train , val, test data
block_size = 3
def build_dataset(words):
  X, Y = [], []
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]
  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

In [17]:
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words)) # 80 %
n2 = int(0.9*len(words)) # 90 %

Xtr, Ytr = build_dataset(words[:n1]) # training data
Xdev, Ydev = build_dataset(words[n1:n2]) # validation data
Xte, Yte = build_dataset(words[n2:]) # testing data

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


> **:)**


In [47]:
# parameters for better nn
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,10), generator=g)
W1 = torch.randn((30,200), generator=g)
b1 = torch.randn(200 , generator=g)
W2 = torch.randn((200,27), generator=g)
b2 = torch.randn(27 , generator=g)
parameters=[C , W1 , b1 , W2 ,b2 ]

In [48]:
sum(p.nelement()  for p in parameters)

11897

In [49]:
for p in parameters:
    p.requires_grad = True

In [42]:
# learning rate Sweep
lre = torch.linspace(-3, 0, 100000)
lrs = 10**lre
lri = [] ; lossi = []; stepi = []

In [55]:
for i in range(10000):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (32,))

  # forward pass
  emb = C[Xtr[ix]] # (32, 3, 10)
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 200)
  logits = h @ W2 + b2 # (32, 27)
  loss = F.cross_entropy(logits, Ytr[ix])
  #print(loss.item())

  # backward pass
  for p in parameters:
    p.grad = None
    loss.backward(retain_graph=True)
  # update lr
  lr = 0.01
  for p in parameters:
    p.data += -lr * p.grad
print(loss.item())

1.740038275718689


In [56]:
emb = C[Xtr] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.2545, grad_fn=<NllLossBackward0>)

In [57]:
emb = C[Xdev] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.2689, grad_fn=<NllLossBackward0>)

In [58]:
context = [0] * block_size
C[torch.tensor([context])].shape

torch.Size([1, 3, 10])

In [59]:
# generate sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    print(''.join(itos[i] for i in out))

mora.
kmyan.
keel.
nah.
yal.
rethan.
ejd.
leg.
azelyn.
elii.
saylona.
edeiseananas.
kayzios.
kalin.
sadbergahiriel.
kinie.
jelilanterian.
brey.
dariyah.
fael.
